# Dzień 2 - Moduł 4: Nowoczesne modele NLP

## Cele modułu:
- Zrozumienie architektury Transformerów
- Poznanie modeli BERT, GPT, T5
- Fine-tuning gotowych modeli na własnych danych
- Praktyczne zastosowanie nowoczesnych modeli

In [ ]:
# Import bibliotek
import torch
import numpy as np
import pandas as pd
from transformers import (
    AutoTokenizer, 
    AutoModel, 
    AutoModelForSequenceClassification,
    pipeline,
    Trainer,
    TrainingArguments
)
from datasets import Dataset
import matplotlib.pyplot as plt
import seaborn as sns

print("✅ Biblioteki załadowane!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## 4.1 Architektura Transformerów i ich wpływ na NLP

### Rewolucja w NLP (2017)
Paper "Attention is All You Need" przedstawił architekturę Transformer, która:
- Zastąpiła rekurencyjne sieci (RNN/LSTM)
- Wprowadził mechanizm uwagi (attention)
- Umożliwił przetwarzanie równoległe
- Skaluje się do miliardów parametrów

### Kluczowe komponenty:

#### 1. Self-Attention (Samo-uwaga)
- Każde słowo "patrzy" na wszystkie inne słowa
- Oblicza ważność (attention score) każdego słowa względem innych
- Pozwala modelować długodystansowe zależności

#### 2. Multi-Head Attention
- Wiele równoległych mechanizmów uwagi
- Każda "głowa" uczy się innych aspektów relacji

#### 3. Positional Encoding
- Koduje pozycję słowa w sekwencji
- Brak tego w oryginalnej architekturze (przetwarzanie równoległe)

#### 4. Feed-Forward Networks
- Warstwy liniowe z aktywacją
- Przetwarzanie każdej pozycji niezależnie

### Dlaczego Transformery wygrały?
- ✅ **Równoległość** - szybszy trening
- ✅ **Długi kontekst** - lepsze rozumienie
- ✅ **Skalowalność** - większe modele = lepsza jakość
- ✅ **Transfer learning** - pre-training + fine-tuning

In [ ]:
# Wizualizacja: Jak działa attention?
# Prosty przykład ilustrujący mechanizm uwagi

def simple_attention_visualization():
    """
    Prosta wizualizacja mechanizmu uwagi.
    """
    # Przykładowe zdanie
    words = ["Kot", "siedzi", "na", "macie"]
    
    # Symulowana macierz uwagi (attention matrix)
    # Każdy wiersz pokazuje jak dane słowo "zwraca uwagę" na inne słowa
    attention_matrix = np.array([
        [0.7, 0.1, 0.1, 0.1],  # "Kot" zwraca uwagę głównie na siebie
        [0.3, 0.5, 0.1, 0.1],  # "siedzi" zwraca uwagę na "Kot" i siebie
        [0.1, 0.2, 0.4, 0.3],  # "na" łączy "siedzi" z "macie"
        [0.2, 0.1, 0.2, 0.5],  # "macie" zwraca uwagę głównie na siebie
    ])
    
    # Wizualizacja
    plt.figure(figsize=(8, 6))
    sns.heatmap(attention_matrix, 
                xticklabels=words, 
                yticklabels=words,
                annot=True, 
                fmt='.2f',
                cmap='YlOrRd',
                cbar_kws={'label': 'Attention Score'})
    plt.title('Attention Matrix - "Kot siedzi na macie"', fontsize=14)
    plt.xlabel('Słowa (Key)')
    plt.ylabel('Słowa (Query)')
    plt.tight_layout()
    plt.show()
    
    print("\n📊 Interpretacja:")
    print("- Jaśniejsze kolory = silniejsza uwaga")
    print("- 'siedzi' zwraca dużą uwagę na 'Kot' (podmiot czasownika)")
    print("- 'na' łączy czasownik z dopełnieniem")

simple_attention_visualization()

## 4.2 Model BERT (Bidirectional Encoder Representations from Transformers)

### Charakterystyka BERT:
- **Bidirectional** - czyta tekst w obu kierunkach jednocześnie
- **Encoder-only** - tylko część enkoder z Transformera
- **Pre-training** - uczony na ogromnych korpusach tekstu
- **Fine-tuning** - łatwo adaptować do konkretnych zadań

### Pre-training BERT:
1. **Masked Language Modeling (MLM)**
   - 15% słów jest maskowanych [MASK]
   - Model próbuje odgadnąć zamaskowane słowa
   - Przykład: "Kot siedzi na [MASK]" → "macie"

2. **Next Sentence Prediction (NSP)**
   - Czy zdanie B następuje po zdaniu A?
   - Pomaga w zadaniach wymagających rozumienia relacji między zdaniami

### Warianty BERT:
- **BERT-base**: 110M parametrów
- **BERT-large**: 340M parametrów
- **RoBERTa**: Ulepszona wersja BERT
- **DistilBERT**: Mniejsza, szybsza wersja
- **HerBERT**: Specjalnie dla języka polskiego

In [ ]:
# Przykład 1: BERT - Fill-Mask (uzupełnianie brakujących słów)
print("=== BERT: FILL-MASK ===")

# Model angielski
fill_mask = pipeline("fill-mask", model="bert-base-uncased")

# Przykłady
examples = [
    "Paris is the [MASK] of France.",
    "The [MASK] is shining brightly in the sky.",
    "I love to [MASK] music.",
]

for example in examples:
    print(f"\nZdanie: {example}")
    results = fill_mask(example, top_k=3)
    print("Top 3 predykcje:")
    for i, result in enumerate(results, 1):
        print(f"  {i}. {result['token_str']:15} (score: {result['score']:.4f})")

In [ ]:
# Przykład 2: BERT Embeddings - reprezentacja wektorowa tekstu
print("\n=== BERT: EMBEDDINGS ===")

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Tekst do zakodowania
texts = [
    "I love natural language processing.",
    "I enjoy working with NLP.",
    "The weather is nice today."
]

# Funkcja do uzyskania embeddingów
def get_bert_embeddings(text):
    # Tokenizacja
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    
    # Forward pass
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Użyj [CLS] token embedding jako reprezentacji całego zdania
    cls_embedding = outputs.last_hidden_state[:, 0, :]
    return cls_embedding

# Uzyskaj embeddingi
embeddings = [get_bert_embeddings(text) for text in texts]

print(f"\nShape embeddingu: {embeddings[0].shape}")
print(f"Każdy tekst reprezentowany przez wektor {embeddings[0].shape[1]} liczb")

# Oblicz podobieństwo cosinusowe
from torch.nn.functional import cosine_similarity

print("\n=== PODOBIEŃSTWA ===")
sim_1_2 = cosine_similarity(embeddings[0], embeddings[1]).item()
sim_1_3 = cosine_similarity(embeddings[0], embeddings[2]).item()
sim_2_3 = cosine_similarity(embeddings[1], embeddings[2]).item()

print(f"Tekst 1 <-> Tekst 2: {sim_1_2:.4f}")
print(f"Tekst 1 <-> Tekst 3: {sim_1_3:.4f}")
print(f"Tekst 2 <-> Tekst 3: {sim_2_3:.4f}")
print("\n💡 Teksty 1 i 2 są najbardziej podobne (oba o NLP)!")

## 4.3 Model GPT (Generative Pre-trained Transformer)

### Charakterystyka GPT:
- **Autoregressive** - generuje tekst słowo po słowie
- **Decoder-only** - tylko część dekoder z Transformera
- **Causal attention** - każde słowo widzi tylko poprzednie słowa
- **Few-shot learning** - może uczyć się z kilku przykładów

### Ewolucja GPT:
- **GPT-1** (2018): 117M parametrów
- **GPT-2** (2019): 1.5B parametrów
- **GPT-3** (2020): 175B parametrów
- **GPT-3.5** (2022): Podstawa ChatGPT
- **GPT-4** (2023): Multimodal, jeszcze potężniejszy

### Pre-training GPT:
- **Causal Language Modeling**
- Przewidywanie następnego słowa
- Przykład: "Kot siedzi na" → "macie"

### BERT vs GPT:

| Aspekt | BERT | GPT |
|--------|------|-----|
| Architektura | Encoder | Decoder |
| Kierunek | Bidirectional | Unidirectional (left-to-right) |
| Pre-training | MLM, NSP | Causal LM |
| Główne zastosowanie | Rozumienie | Generowanie |
| Kontekst | Całe zdanie | Poprzednie słowa |

In [ ]:
# Przykład: GPT-2 - generowanie tekstu
print("=== GPT-2: GENEROWANIE TEKSTU ===")

generator = pipeline("text-generation", model="gpt2")

prompts = [
    "Artificial intelligence will",
    "In the future, technology",
    "Natural language processing is",
]

for prompt in prompts:
    print(f"\n{'='*60}")
    print(f"Prompt: {prompt}")
    print("-" * 60)
    
    results = generator(
        prompt,
        max_length=50,
        num_return_sequences=2,
        temperature=0.8,  # Kontrola "kreatywności" (0 = deterministyczne, 1+ = kreatywne)
        do_sample=True
    )
    
    for i, result in enumerate(results, 1):
        print(f"\nWersja {i}:")
        print(result['generated_text'])

## 4.4 Model T5 (Text-to-Text Transfer Transformer)

### Filozofia T5:
- **Wszystko jako Text-to-Text**
- Każde zadanie NLP to przekształcenie tekstu na tekst
- Uniwersalny format treningowy

### Przykłady zadań:
- **Tłumaczenie**: "translate English to German: Hello" → "Hallo"
- **Podsumowanie**: "summarize: [długi tekst]" → [krótki tekst]
- **Klasyfikacja**: "sentiment: This is great!" → "positive"

### Architektura:
- **Encoder-Decoder** (pełny Transformer)
- Połączenie zalet BERT i GPT
- Encoder rozumie, Decoder generuje

### Warianty:
- **T5-small**: 60M parametrów
- **T5-base**: 220M parametrów
- **T5-large**: 770M parametrów
- **T5-3B/11B**: Największe wersje

In [ ]:
# Przykład: T5 - różne zadania
print("=== T5: TEXT-TO-TEXT ===")

# Podsumowanie
summarizer = pipeline("summarization", model="t5-small")

long_text = """
Natural language processing (NLP) is a subfield of linguistics, computer science, 
and artificial intelligence concerned with the interactions between computers and 
human language. It involves programming computers to process and analyze large 
amounts of natural language data. Challenges in NLP frequently involve speech 
recognition, natural language understanding, and natural language generation. 
Modern NLP techniques are based on machine learning, especially deep learning.
"""

print("\n--- PODSUMOWANIE ---")
summary = summarizer(long_text, max_length=50, min_length=20)
print(f"Oryginał ({len(long_text.split())} słów):")
print(long_text.strip())
print(f"\nPodsumowanie:")
print(summary[0]['summary_text'])

In [ ]:
# T5 - Tłumaczenie (jeśli dostępny model)
try:
    translator = pipeline("translation_en_to_de", model="t5-small")
    
    print("\n--- TŁUMACZENIE (EN -> DE) ---")
    english_text = "Hello, how are you today?"
    translation = translator(english_text)
    print(f"English: {english_text}")
    print(f"German: {translation[0]['translation_text']}")
except Exception as e:
    print(f"\n⚠️ Tłumaczenie niedostępne: {e}")

## 4.5 Fine-tuning gotowych modeli na własnych danych

### Co to jest Fine-tuning?
- Dostosowanie pre-trenowanego modelu do konkretnego zadania
- Dużo szybsze i tańsze niż trening od zera
- Wymaga mniej danych niż trening od podstaw

### Kroki fine-tuningu:
1. Wybór pre-trenowanego modelu
2. Przygotowanie danych treningowych
3. Konfiguracja parametrów treningu
4. Trening (fine-tuning)
5. Ewaluacja
6. Zapisanie i wdrożenie modelu

### Kiedy fine-tunować?
- ✅ Masz specyficzną domenę (medycyna, prawo, finanse)
- ✅ Masz etykietowane dane treningowe
- ✅ Ogólne modele nie dają wystarczającej jakości
- ✅ Potrzebujesz specyficznego formatu wyjścia

In [ ]:
# Przykład: Fine-tuning BERT do klasyfikacji sentymentu
print("=== FINE-TUNING: KLASYFIKACJA SENTYMENTU ===")

# 1. Przygotowanie danych
train_data = [
    {"text": "This product is amazing! I love it!", "label": 1},
    {"text": "Terrible experience. Never buying again.", "label": 0},
    {"text": "Great quality and fast shipping.", "label": 1},
    {"text": "Worst purchase ever. Completely broken.", "label": 0},
    {"text": "Absolutely fantastic! Highly recommend!", "label": 1},
    {"text": "Poor quality. Very disappointed.", "label": 0},
    {"text": "Excellent product. Worth every penny.", "label": 1},
    {"text": "Waste of money. Don't buy this.", "label": 0},
]

# Konwersja do Dataset
dataset = Dataset.from_list(train_data)

print("Dataset:")
print(dataset)
print(f"\nPrzykład: {dataset[0]}")

In [ ]:
# 2. Przygotowanie modelu i tokenizera
model_name = "distilbert-base-uncased"  # Mniejszy, szybszy BERT
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, 
    num_labels=2  # binary classification
)

print(f"\n✅ Model załadowany: {model_name}")
print(f"Parametry: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# 3. Tokenizacja danych
def tokenize_function(examples):
    return tokenizer(
        examples["text"], 
        padding="max_length", 
        truncation=True,
        max_length=128
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True)

print("\n✅ Dane ztokenizowane")
print(f"Kolumny: {tokenized_dataset.column_names}")

In [ ]:
# 4. Konfiguracja treningu
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    warmup_steps=10,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="no",  # Nie zapisuj checkpointów (dla demonstracji)
)

# 5. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

print("\n✅ Trainer skonfigurowany")
print("\n⚠️ W prawdziwym projekcie teraz uruchomilibyśmy: trainer.train()")
print("(Pomijamy trening w demonstracji ze względu na czas)")

# W prawdziwym scenariuszu:
# trainer.train()
# model.save_pretrained("./my_finetuned_model")

In [ ]:
# 6. Testowanie (z pre-trenowanym modelem)
# Użyjmy gotowego fine-tunowanego modelu do sentymentu
print("\n=== TESTOWANIE FINE-TUNOWANEGO MODELU ===")

sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english"
)

test_texts = [
    "This is absolutely wonderful!",
    "I hate this product.",
    "It's okay, nothing special.",
]

for text in test_texts:
    result = sentiment_pipeline(text)[0]
    print(f"\nText: {text}")
    print(f"  → {result['label']}: {result['score']:.4f}")

## Ćwiczenie praktyczne

### Zadanie:
1. Wybierz pre-trenowany model (BERT, GPT, T5)
2. Użyj go do konkretnego zadania:
   - BERT: Klasyfikacja lub ekstrakcja informacji
   - GPT: Generowanie tekstu
   - T5: Podsumowanie lub tłumaczenie
3. Eksperymentuj z różnymi parametrami
4. Porównaj wyniki różnych modeli

In [ ]:
# MIEJSCE NA TWÓJ KOD
# Eksperymentuj z różnymi modelami!

print("💡 Wskazówki:")
print("- Użyj Hugging Face Model Hub: https://huggingface.co/models")
print("- Szukaj modeli dla swojego języka (np. 'polish bert')")
print("- Sprawdź dokumentację modelu przed użyciem")
print("- Eksperymentuj z parametrami (temperature, max_length, etc.)")

## Podsumowanie Modułu 4

✅ Zrozumieliśmy architekturę Transformerów i mechanizm attention

✅ Poznaliśmy główne modele: BERT (rozumienie), GPT (generowanie), T5 (text-to-text)

✅ Nauczyliśmy się używać pre-trenowanych modeli

✅ Poznaliśmy proces fine-tuningu własnych modeli

✅ Wykonaliśmy praktyczne przykłady z każdym typem modelu

### Kluczowe różnice:

| Model | Architektura | Zastosowanie | Kierunek |
|-------|-------------|--------------|----------|
| BERT | Encoder | Rozumienie, klasyfikacja | Bidirectional |
| GPT | Decoder | Generowanie | Unidirectional |
| T5 | Encoder-Decoder | Uniwersalne | Both |

---

**Następny krok**: Moduł 5 - Generowanie i rozumienie tekstu w praktyce